In [1]:
#Get the dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
#Get Input Text
with open('input.txt', 'r') as f:
    text = f.read()

In [3]:
#Get All Unique Characters
chars = sorted(list(set(text)))
print(chars)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
#Tokenize the text- This will be used to convert the text to numbers
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

#Encode and Decode Functions for Strings
def encode(s):
    return [char_to_idx[ch] for ch in s]
def decode(l):
    return ''.join([idx_to_char[i] for i in l])

#Test
encodedText = encode("Hello!")
print(encodedText)
print(decode(encodedText))

#Note: This is a very simple tokenization method. There are better ways to do this. Example: Using tiktoken from OpenAI. (Sub-word tokenization = Don't need a new token for every letter)


[20, 43, 50, 50, 53, 2]
Hello!


In [19]:
#Some Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

In [7]:
#Encode the input text and store it into a Torch Tensor
#A tensor is a multi-dimensional matrix containing elements of a single data type

#Convert the text to numbers
encodedText = encode(text)
print("Length of encoded text: ", len(encodedText))

#Convert the list to a tensor
data = torch.tensor(encodedText, dtype=torch.long)
print(data.shape)
print(data[:10]) #First 10 elements (First 10 characters in the text)

Length of encoded text:  1115394
torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [8]:
#Get Training and Validation Data
#We will use the first 90% of the data for training and the last 10% for validation

#Get the length of the data
dataLen = len(data)
print("Length of data: ", dataLen)

#Get the length of the training data  
trainLen = int(dataLen * 0.9)

#Get the training data
train_data = data[:trainLen]
print("Length of training data: ", len(train_data))

#Get the validation data
val_data = data[trainLen:]  
print("Length of validation data: ", len(val_data))

Length of data:  1115394
Length of training data:  1003854
Length of validation data:  111540


In [16]:
block_size = 8 #The length of the sequence we want to train on (the context)
#Example: If block_size = 8, then we want to predict the 9th character in the sequence
#This is why we will need to train the model on sequences of length block_size + 1
#In the below code the max value of t+1 is block_size

x = train_data[0:block_size] # The input sequence
y = train_data[1:block_size+1] # The output sequence

print("Input sequence: ", x)
print("Output sequence: ", y)

for t in range(block_size): 
    context = x[:t+1]
    target = y[t]
    print(t+1, ': Context: ', str(context), ' Target: ', str(target)) 
    #Ex. If data is {1,2,3} and we are given 1, then the target (predicted output) should be 2 

Input sequence:  tensor([18, 47, 56, 57, 58,  1, 15, 47])
Output sequence:  tensor([47, 56, 57, 58,  1, 15, 47, 58])
1 : Context:  tensor([18])  Target:  tensor(47)
2 : Context:  tensor([18, 47])  Target:  tensor(56)
3 : Context:  tensor([18, 47, 56])  Target:  tensor(57)
4 : Context:  tensor([18, 47, 56, 57])  Target:  tensor(58)
5 : Context:  tensor([18, 47, 56, 57, 58])  Target:  tensor(1)
6 : Context:  tensor([18, 47, 56, 57, 58,  1])  Target:  tensor(15)
7 : Context:  tensor([18, 47, 56, 57, 58,  1, 15])  Target:  tensor(47)
8 : Context:  tensor([18, 47, 56, 57, 58,  1, 15, 47])  Target:  tensor(58)


In [17]:
torch.manual_seed(1337) #Set the seed for reproducibility
block_size = 8 #The length of the sequence we want to train on 
batch_size = 4 #The number of sequences we want to train on at a time

def get_batch(split):
    #Get the data depending on the split (train or validation)
    if split == 'train':
        data = train_data
    else:
        data = val_data
    
    #Get batch_size random starting indexes
    ix = torch.randint(high=len(data)-block_size, size=(batch_size,))

    #Construct the input and output sequences
    x = [data[i:i+block_size] for i in ix]
    y = [data[i+1:i+block_size+1] for i in ix]
    
    #Convert the sequences to tensors of size (batch_size, block_size)
    x = torch.stack(x)
    y = torch.stack(y)

    return x, y

#Test
xb, yb = get_batch('train')
print("Input batch shape: ", xb.shape)
print("Output batch shape: ", yb.shape)

print("-----------------")
print("Input batch: ")
print(xb)
print("-----------------")
print("Output batch: ")
print(yb)
print("-----------------")

for t in range(block_size): #Basically the same as the previous but now we are using batches (multiple sequences)
    context = xb[:, :t+1] #Get the first t+1 elements of each sequence
    target = yb[:, t]
    print(t+1, ': Context: ', str(context), ' Target: ', str(target)) 
    #Ex. If data is {1,2,3} and we are given 1, then the target (predicted output) should be 2

Input batch shape:  torch.Size([4, 8])
Output batch shape:  torch.Size([4, 8])
-----------------
Input batch: 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
-----------------
Output batch: 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------
1 : Context:  tensor([[24],
        [44],
        [52],
        [25]])  Target:  tensor([43, 53, 58, 17])
2 : Context:  tensor([[24, 43],
        [44, 53],
        [52, 58],
        [25, 17]])  Target:  tensor([58, 56,  1, 27])
3 : Context:  tensor([[24, 43, 58],
        [44, 53, 56],
        [52, 58,  1],
        [25, 17, 27]])  Target:  tensor([ 5,  1, 58, 10])
4 : Context:  tensor([[24, 43, 58,  5],
        [44, 53, 56,  1],
        [52, 58,  1, 58],
        [25, 17, 27, 10]])  Target:  tenso

In [22]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        #super() returns the parent object of the class
        #In this case, it returns the parent object of BigramLanguageModel which is nn.Module (defined above)
        super().__init__() # Call the parent class constructor

        
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #Embedding layer
        #Embedding layer is used to convert the input tokens to vectors of size embedding_dim
        #In this case, the input tokens are numbers (the encoded text) and the embedding_dim is vocab_size
        #Vectors will allow us to do math operations on the tokens

    def forward(self, idx, targets):

        logits = self.token_embedding_table(idx) #Get the embeddings for the input tokensA
        return logits

m = BigramLanguageModel(len(chars)) #Create the model
out = m(xb, yb) #Get the output
print(out.shape) #Print the shape of the output
        


torch.Size([4, 8, 65])


In [ ]:
import t